### k-Fold Cross Validation
Split dataset into k parts (folds)

Train on k-1, test on 1

Repeat k times with different test folds

Example: 5-Fold CV
→ Train on 4 folds, test on 1 → Repeat 5 times

In [ ]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Load sample classification dataset
X, y = load_iris(return_X_y=True)
model = RandomForestClassifier()

# List of metrics we want to evaluate
metrics = ["accuracy", "precision_macro", "recall_macro", "f1_macro"]

# Loop over metrics and compute cross-validation scores
for metric in metrics:
    scores = cross_val_score(model, X, y, cv=5, scoring=metric)
    print(f"\n🔹 Metric: {metric}")
    print(f"Fold Scores: {scores}")
    print(f"Mean: {scores.mean():.4f}, Std Dev: {scores.std():.4f}")


In [ ]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split

# Load example dataset
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = RandomForestClassifier()

# Cross-validation check
cv_scores = cross_val_score(model, X_train, y_train, cv=5)
print("CV scores:", cv_scores)

# Overfitting check
model.fit(X_train, y_train)
train_acc = model.score(X_train, y_train)
test_acc = model.score(X_test, y_test)
print(f"Train acc: {train_acc:.2f}, Test acc: {test_acc:.2f}")

if abs(train_acc - test_acc) > 0.15:
    print("Overfitting Detected")
else:
    print("No major overfitting")


### Data Leakage Detection (Manual Feature Audit)

In [ ]:
import pandas as pd
import numpy as np

# Simulate leaky data
df = pd.DataFrame({
    "age": [30, 45, 25, 60],
    "blood_pressure": [120, 130, 110, 140],
    "diagnosis": [0, 1, 0, 1]
})
df["leaky_feature"] = df["diagnosis"] * 10  # 🚨 Simulated leakage

# Detect leakage by checking correlation with target
correlations = df.corr(numeric_only=True)
print(correlations["diagnosis"].sort_values(ascending=False))

if any(abs(correlations["diagnosis"]) > 0.95):
    print("Potential leakage detected (very high correlation)")


### Drift Detection with KS-Test (Data Drift)

In [ ]:
from scipy.stats import ks_2samp

# Simulated old (train) and new (live) data
train_feature = [1, 2, 2, 3, 4, 4, 5]
live_feature = [6, 6, 7, 8, 8, 9, 9]

# KS-test for distribution change
stat, p_value = ks_2samp(train_feature, live_feature)
print("KS p-value:", p_value)

if p_value < 0.05:
    print("Drift detected")
else:
    print("No significant drift")


### Regression vs Classification Evaluation

In [ ]:
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))


### Regression (MAE, R²)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

# Regression example
X = np.random.rand(100, 1) * 10
y = 3 * X.squeeze() + np.random.randn(100)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
reg_model = LinearRegression().fit(X_train, y_train)
y_pred = reg_model.predict(X_test)

print("MAE:", mean_absolute_error(y_test, y_pred))
print("R2 Score:", r2_score(y_test, y_pred))


### Latency Monitoring for GenAI

In [ ]:
import time

def fake_genai_call(prompt):
    time.sleep(0.9)  # simulate latency
    return "response"

start = time.time()
output = fake_genai_call("What is the attention mechanism?")
end = time.time()

latency = end - start
print(f"Latency: {latency:.2f} seconds")

if latency > 1.0:
    print("GenAI call too slow")
else:
    print("GenAI latency within limit")


In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Load dataset
X, y = load_iris(return_X_y=True)
model = RandomForestClassifier()

# Define metrics
metrics = ["accuracy", "precision_macro", "recall_macro", "f1_macro"]
metric_labels = ["Accuracy", "Precision", "Recall", "F1 Score"]
fold_scores = []

# Collect fold scores for each metric
for metric in metrics:
    scores = cross_val_score(model, X, y, cv=5, scoring=metric)
    fold_scores.append(scores)

# Transpose for plotting: fold_scores[i][j] → metric i, fold j
fold_scores = list(zip(*fold_scores))  # Now: each inner list = metrics for fold j

# --- Visualization ---
folds = range(1, 6)
bar_width = 0.2

plt.figure(figsize=(10, 6))
for i, metric_name in enumerate(metric_labels):
    values = [fold[i] for fold in fold_scores]
    plt.bar([f + i*bar_width for f in folds], values, width=bar_width, label=metric_name)

plt.xlabel("Fold Number")
plt.ylabel("Score")
plt.title("Cross-Validation Scores Across Folds")
plt.xticks([f + 1.5*bar_width for f in folds], [f"Fold {i}" for i in folds])
plt.ylim(0.85, 1.05)
plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Create imbalanced dataset (90% of class 0, 10% of class 1)
X, y = make_classification(
    n_samples=1000, n_classes=2,
    weights=[0.9, 0.1], flip_y=0,
    n_features=10, random_state=42
)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3)

# Model
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Pred 0", "Pred 1"], yticklabels=["True 0", "True 1"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# Classification Report
print("\n", classification_report(y_test, y_pred))
